<a href="https://colab.research.google.com/github/Mozzer2310/COMP34212-Notebooks/blob/main/COMP34212_Lab3_RNN_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Recurrent Neural Networks (RNN): Simple RNNs and LSTMs
=========

In this tutorial we will learn how to use Recurremt Neural Networks (RNNs) for text processing.
The first example uses a simple RNN to generate the characters in a small text corpus (Alice in Wonderland novel).
We will then look at the Long Short Term Memory (LSTM) networks, for more advanced recurrent models. The LSTM will be tested on a sentiment analysis corpus.


1 - Simple RNNs to Generate text
=========

This exmple will use a simple RNN with one layer of hidden and recurrent units. 

Let's first import the usual keras and utility functions. This will include the first importing and use of the SimpleRNN recurrent layer type in Keras.


In [ ]:
from __future__ import print_function
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.models import Sequential
import numpy as np


**Loading text file and converting to clean text**

This code will read the file "alice_in_wonderland.txt". This file is available for download here http://www.gutenberg.org/files/11/11-0.txt (and can also be downloaded from Balckboard ). 

The code will do some preliminary cleanup of the text (e.g. removing non-ASCII characters and line breaks) and write all words in the variable called "whole_text".


In [ ]:
print("Loading text file...")

INPUT_FILE = "alice_in_wonderland.txt"

fin = open(INPUT_FILE, 'rb')
lines = []
for line in fin:
    line = line.strip().lower()
    line = line.decode("ascii", "ignore")
    if len(line) == 0:
        continue
    lines.append(line)
fin.close()
whole_text = " ".join(lines)

print("Done!")


Loading text file...
Done!


**Characters look-up table**

This code will create the look-up table from the 42 characters to integer IDs, and viceversa. 

In [ ]:
print("Preparing characters look-up table...")

chars = set([c for c in whole_text])
nb_chars = len(chars)
char2index = dict((c, i) for i, c in enumerate(chars))
index2char = dict((i, c) for i, c in enumerate(chars))

print("Done!")

Preparing characters look-up table...
Done!


**Input text and output character**

To create the input text, the code will step through the whole text by a number of characters defined by the __STEP__ variable (1 in our case) and then extract a set of characters whose size is determined by the __SEQLEN__ variable (10 in our case). The next character after the extracted characters is the output label, i,e. the next character to predict.

Here is an example of the input/output for the part of text starting as" it turned into a pig" 

   INPUT (10)    ->   OUTPUT (1)
- "it turned "   ->   i
- "t turned i"   ->   n
- " turned in"   ->   t
- "turned int"   ->   o
- "urned into"   ->
- "rned into "   ->   a
- "ned into a"   ->
- "ed into a "   ->   p
- "d into a p"   ->   i
- " into a pi"   ->   g


In [ ]:
print("Creating input and label text...")

SEQLEN = 10
STEP = 1

input_chars = []
label_chars = []
for i in range(0, len(whole_text) - SEQLEN, STEP):
    input_chars.append(whole_text[i:i + SEQLEN])
    label_chars.append(whole_text[i + SEQLEN])

print("Done!")

Creating input and label text...
Done!


**Vectorisation of the input and output**

This preprares the input and output vectors of the training set. 
The input vector uses one-hot encoding of the __SEQLEN__ (10) characters present in the input text segment, times the __nb_chars__ (42) possible characters.
The output label uses one-hot encoding for the activation of a single character out of the __nb_chars__ number of units/characters.


In [ ]:
print("Vectorizing input and label text...")

X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=bool)
y = np.zeros((len(input_chars), nb_chars), dtype=bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

print("Done!")

Vectorizing input and label text...


<ipython-input-5-4c1b68e28321>:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
<ipython-input-5-4c1b68e28321>:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)


Done!


**Simple RNN Model definition**

This codes defines the key variables of the model and training, and creates the sequential model. The model consists of a simple recurrent neural network (RNN) with a hidden layer of 128 simple recurrent units.

The __return_sequences__ is set to __False__ as the output only consists of one character, and not a sequence. 
The __unroll=True__ setting improves performance on the TensorFlow backend.
The optimiser uses the __rmsprop__ for backpropagation.



In [ ]:
# Definition of the network and training hyperparameters
HIDDEN_SIZE = 128
BATCH_SIZE = 128
NUM_ITERATIONS = 25
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

# Definition of the network topology, with simpleRNN hidden layer
model = Sequential()
model.add(SimpleRNN(HIDDEN_SIZE, return_sequences=False, input_shape=(SEQLEN, nb_chars), unroll=True))
model.add(Dense(nb_chars))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

#show the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 128)               23424     
                                                                 
 dense (Dense)               (None, 54)                6966      
                                                                 
 activation (Activation)     (None, 54)                0         
                                                                 
Total params: 30,390
Trainable params: 30,390
Non-trainable params: 0
_________________________________________________________________


**Training and testing of the model**

The model is trained for __NUM_ITERATIONS__ (25) epochs and tested after each epoch, to allow us to monitor the improvement of the model performance in character prediction.

The test consists of generating a character from the model given a random input, then dropping the first character from the input and appending the predicted character from the previous run as the new input, to generate another character. This is done for __NUM_PREDS_PER_EPOCH__ (100) steps. The completed string gives us an indication of the quality of the model's processing of English words (within the limited lexicon of Alice's novel).


In [ ]:
for iteration in range(NUM_ITERATIONS):
    print("=" * 50)
    print("Iteration #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)
    
    # testing model
    # randomly choose a row from input_chars, then use it to 
    # generate text from model for next 100 chars
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]
    print("Generating from seed: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):
        Xtest = np.zeros((1, SEQLEN, nb_chars))
        for i, ch in enumerate(test_chars):
            Xtest[0, i, char2index[ch]] = 1
        pred = model.predict(Xtest, verbose=0)[0]
        ypred = index2char[np.argmax(pred)]
        print(ypred, end="")
        # move forward with test_chars + ypred
        test_chars = test_chars[1:] + ypred
    print()


Iteration #: 0
1236/1236 [==============================] - 42s 31ms/step - loss: 2.3404
Generating from seed: th is it? 
th is it? alice and the the sore the the sore the the sore the the sore the the sore the the sore the the sore
Iteration #: 1
1236/1236 [==============================] - 27s 22ms/step - loss: 2.0425
Generating from seed: n of cours
n of cours and and the said the said the said the said the said the said the said the said the said the said t
Iteration #: 2
1236/1236 [==============================] - 26s 21ms/step - loss: 1.9353
Generating from seed: g--catch h
g--catch hares and the said the cateres and alice there the said the cateres and alice there the said the cate
Iteration #: 3
1236/1236 [==============================] - 18s 14ms/step - loss: 1.8550
Generating from seed: use out of
use out of the said the doon the said the doon the said the doon the said the doon the said the doon the said 
Iteration #: 4
1236/1236 [==============================] - 16s 13ms

This example shows how we can train a Simple RNN to predict the next charaters, using the information from the sequentail relationship between words in the training text.
You can try the same code on a different, longer text, or try different hyperparameters.

We are now ready to try a more complex RNN architecture, for more complex time series tasks.


2 - LSTM for Sentiment Analysis
=========

This example will show how to code the more complex RNN model of the Long Short Term Memmory (LSTM) network. This tutorial will use the example of sentiment analysis prediction.

Let's start with the usual importing of Keras and utility modules. This will of course include the LSTM recurrent layer type.


In [ ]:
from tensorflow.keras.layers import Activation, Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import collections
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt') # download instruction required if pubkt nltk package not installed
import numpy as np
import os

print("Initialisation Done! Make sure you have initialised ntlk punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Initialisation Done! Make sure you have initialised ntlk punkt


Let's now define the main hyperparameter values and text corpus details.


In [ ]:
INPUT_FILE = "umich-sentiment-train.txt"

EMBEDDING_SIZE = 128
HIDDEN_LAYER_SIZE = 64
BATCH_SIZE = 32
NUM_EPOCHS = 10


**Analysis of words in corpus**

This code load the text from the UMICH labelled sentiment analysis corpus file, to identify the number of unique words (length of the __word_freq__ variable, i.e. 2313) and the max number of words in the longest sentence (__maxlen__ expected to be 42). These two variables will be needed later for the preaparation of the training set.


In [ ]:
# Read training data and generate vocabulary
maxlen = 0
word_freqs = collections.Counter()
num_recs = 0

ftrain = open(INPUT_FILE, 'rb')
i = 0
for line in ftrain:
    label, sentence = line.strip().split(b"\t")
    words = nltk.word_tokenize(sentence.decode("ascii", "ignore").lower())
    if len(words) > maxlen:
        maxlen = len(words)
    for word in words:
        word_freqs[word] += 1
    num_recs += 1
    i += 1
ftrain.close()

## Get some information about our corpus
print (maxlen)            # 42
print (len(word_freqs))   # 2313

FileNotFoundError: ignored

**Unkown word and padding word and lookup tables**

Knowing the number of unique words will permit the identification of the out of vocabulary (OOV) words, which can be replaced by the pseudo-word "UNK" (for unknown). When testing the network prediction, this will allow us to handle previously unseen OOV words.

We can decide a fixed sequence length by truncating longer sentences to that length as appropriate (__MAX_SENTENCE_LENGTH = 40__). We also use the PAD to make shorter sentence equivalent to the fixed max length.

Overall, we set our VOCABULARY_SIZE to 2002. i.e. 2,000 words from our vocabulary plus the UNK pseudo-word and the PAD pseudo word used for padding sentences up to the max length.

We then create the look-up tables. Each row of input to the network is a sequence of word indices, where the indices are ordered by most frequent to least frequent word in the training set.

In [ ]:
MAX_FEATURES = 2000
MAX_SENTENCE_LENGTH = 40

# 1 is UNK, 0 is PAD
# We take MAX_FEATURES-1 featurs to accound for PAD
vocab_size = min(MAX_FEATURES, len(word_freqs)) + 2
word2index = {x[0]: i+2 for i, x in 
                enumerate(word_freqs.most_common(MAX_FEATURES))}
word2index["PAD"] = 0
word2index["UNK"] = 1
index2word = {v:k for k, v in word2index.items()}


**Prepare training sequences**

The input sentences need to be converted to  word index sequences (inlcuding their padiing to the max sequecne length). 
No procesing required for the output, as this is a positive/negative binary sentiment output.


In [ ]:
# convert sentences to sequences
X = np.empty((num_recs, ), dtype=list)
y = np.zeros((num_recs, ))
i = 0
ftrain = open(INPUT_FILE, 'rb')
for line in ftrain:
    label, sentence = line.strip().split(b"\t")
    words = nltk.word_tokenize(sentence.decode("ascii", "ignore").lower())
    seqs = []
    for word in words:
        if word in word2index:
            seqs.append(word2index[word])
        else:
            seqs.append(word2index["UNK"])
    X[i] = seqs
    y[i] = int(label)
    i += 1
ftrain.close()

# Pad the sequences (left padded with zeros)
X = sequence.pad_sequences(X, maxlen=MAX_SENTENCE_LENGTH)

# Split input into training and test
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42)
print(Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape)

**Model building and compilation**

The codes defines the LSTM model topology, as per figure below. This is a stdnard recurrent network with one layer of LSTM units.

![image.png](attachment:image.png)


In [ ]:
# Build model
model = Sequential()
model.add(Embedding(vocab_size, EMBEDDING_SIZE, input_length=MAX_SENTENCE_LENGTH))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(HIDDEN_LAYER_SIZE, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1))
model.add(Activation("sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

#show the model summary
model.summary()


**Training of the model**

Let's train our LSTM RNN.
The model uses the binary cross-entropy loss function, suitable for binary value prediction. The Adam optimizer is one of the best and most frequently used general purpose optimizers.


In [ ]:
history = model.fit(Xtrain, ytrain, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, validation_data=(Xtest, ytest))

**Plotting of the results**

The code below will produce accuracy and loss plots for training and validation data.


In [ ]:
# plot loss and accuracy
plt.subplot(211)
plt.title("Accuracy")
plt.plot(history.history["accuracy"], color="g", label="Train")
plt.plot(history.history["val_accuracy"], color="b", label="Validation")
plt.legend(loc="best")

plt.subplot(212)
plt.title("Loss")
plt.plot(history.history["loss"], color="g", label="Train")
plt.plot(history.history["val_loss"], color="b", label="Validation")
plt.legend(loc="best")

plt.tight_layout()
plt.show()


**Evaluation of the model**

The model is evaluated against the full test set, printing the score and accuracy. 
The code also generates a few random sentences from the test set and prints the model's prediction, the label and the actual sentence. This will give us an idea of the quality of the prediction.


In [ ]:
# evaluate
score, acc = model.evaluate(Xtest, ytest, batch_size=BATCH_SIZE)
print("Test score: %.3f, accuracy: %.3f" % (score, acc))

for i in range(5):
    idx = np.random.randint(len(Xtest))
    xtest = Xtest[idx].reshape(1,40)
    ylabel = ytest[idx]
    ypred = model.predict(xtest)[0][0]
    sent = " ".join([index2word[x] for x in xtest[0].tolist() if x != 0])
    print("%.0f\t%d\t%s" % (ypred, ylabel, sent))


Conclusions
-------------

This exercise finishes our short deep leanring course. You know have a pracyicla understanding of how to train a simple MLP (multi-Layer Perceptron), CNNs (Concolutional Neural Networks) with different layers, a simple RNN (Recurrent Neural network) and the more complex LSTM (Long Short Term Neural Network). Well done!

**Copyright (c)** 2022 Angelo Cangelosi, MIT License. Code and examples adapted from Gulli & Pal (2017) Deep Learning with Keras. Punkt Publishing. With the support of Wenjie Huang.